## Predicting Concrete Compressive Strength Using Deep Learning: A Regression Approach with Keras

In [ ]:
# All Libraries required for this lab are listed below.

!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0

In [ ]:
import pandas as pd
import numpy as np
import keras

In [ ]:
filepath='data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

In [ ]:
concrete_data.shape

In [ ]:
concrete_data.describe()


In [ ]:
concrete_data.isnull().sum()

In [7]:
concrete_data_columns = concrete_data.columns

In [8]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [ ]:
predictors.head()

In [ ]:
target.head()

In [ ]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

##  Import Keras Packages

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

## Build a Neural Network

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))  # Input layer
    model.add(Dense(10, activation='relu'))  # One hidden layer with 10 nodes and ReLU activation
    model.add(Dense(1))  # Output layer for regression (single output node)
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')  # Adam optimizer and MSE loss function
    return model

## Train and Test the Network

In [17]:
# build the model
baseline_model = regression_model()

In [ ]:
# fit the model
baseline_model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

## Randomly split the data into a training and test sets by holding 30% of the data for testing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

# Fit the model on the training data
baseline_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=2)

In [23]:
# Function to train and evaluate the model
def train_and_evaluate():
    # Split the data into training and test sets (30% test set)
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=None)
    
    # Build the model
    model = regression_model()
    
    # Train the model on the training data
    model.fit(X_train, y_train, epochs=50, verbose=0)  # Train for 50 epochs
    
    # Predict on the test set
    predictions = model.predict(X_test)
    
    # Compute the mean squared error
    mse = mean_squared_error(y_test, predictions)
    return mse

In [ ]:
# Repeat the process 50 times
mse_list = [train_and_evaluate() for _ in range(50)]

# Compute the mean and standard deviation of the MSEs
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

# Report the results
print(f"Mean of Mean Squared Errors: {mean_mse}")
print(f"Standard Deviation of Mean Squared Errors: {std_mse}")

## Normalize the data 